In [1]:
import os
import re
import json
import torch
from uuid import uuid4
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels
from qdrant_client.models import PointStruct, VectorParams, Distance
from transformers import AutoTokenizer, AutoModel

In [2]:
MODEL_NAME = "intfloat/multilingual-e5-large"
COLLECTION_NAME = os.getenv("COLLECTION_NAME")
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).eval()

In [4]:
# Функция для создания эмбеддингов
def embed_chunk(text):
    return model.encode(f"passage: {text}", normalize_embeddings=True)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [7]:
# Разделение на чанки
def chunk_text(text, max_sentences=3):
    text = text.replace("\t", " ")
    text = re.sub(r'\s+', ' ', text).strip()
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    chunks = []
    for i in range(0, len(sentences), max_sentences):
        chunk = " ".join(sentences[i:i+max_sentences])
        if len(chunk) > 0:
            chunks.append(chunk)
    return chunks

In [8]:
# Рекурсивных обход словарей и списков
def extract_chunks(obj, program_name, path=""):
    chunks = []
    if isinstance(obj, str):
        if re.match(r'^https?://', obj):
            return []
        for chunk in chunk_text(obj):
            full_text = f"{path}: {chunk}" if path else chunk
            chunks.append({
                "раздел": path,
                "текст": full_text,
                "program": program_name})
    elif isinstance(obj, list):
        for i, item in enumerate(obj):
            chunks += extract_chunks(item, program_name, f"{path}[{i}]")
    elif isinstance(obj, dict):
        for key, value in obj.items():
            new_path = f"{path}.{key}" if path else key
            chunks += extract_chunks(value, program_name, new_path)
    return chunks
    
# Обход всех образотательных программ
def load_all_program_chunks(folder_path="itmo_data"):
    all_chunks = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            filepath = os.path.join(folder_path, filename)
            try:
                with open(filepath, "r", encoding="utf-8") as f:
                    data = json.load(f)
                if isinstance(data, list):
                    for item in data:
                        program_name = (
                            item.get('program', {}).get('Название программы', 'unknown_program')
                            if isinstance(item, dict) else 'unknown_program')
                        chunks = extract_chunks(item, program_name)
                        all_chunks.extend(chunks)
                elif isinstance(data, dict):
                    program_name = data.get('program', {}).get('Название программы', 'unknown_program')
                    chunks = extract_chunks(data, program_name)
                    all_chunks.extend(chunks)
                else:
                    print(f"❗️Неожиданный тип данных в файле: {filename}")
            except Exception as e:
                print(f"❌ Ошибка при обработке {filename}: {e}")
    return all_chunks

In [9]:
all_chunks = load_all_program_chunks("itmo_data")
print(f"Всего чанков: {len(all_chunks)}")

Всего чанков: 3811


In [10]:
for chunk in all_chunks:
    chunk["embedding"] = embed_chunk(chunk["текст"]).tolist()

In [11]:
# Подготовка всех PointStruct
points = [
    PointStruct(
        id=uuid.uuid4().hex,
        vector=chunk["embedding"],
        payload={
            "program": chunk["program"].lower(),  # название программы вынесено в качестве тега фильтрации
            "раздел": chunk["раздел"],
            "текст": chunk["текст"]
        }
    )
    for chunk in all_chunks
]

# Функция для разбиения на батчи
def batch(iterable, batch_size):
    for i in range(0, len(iterable), batch_size):
        yield iterable[i:i + batch_size]

# Загрузка по 200 штук
for i, batch_points in enumerate(batch(points, 200), 1):
    client.upsert(collection_name=COLLECTION_NAME, points=batch_points)
    print(f"Загружен батч {i} ({len(batch_points)} точек)")

Загружен батч 1 (200 точек)
Загружен батч 2 (200 точек)
Загружен батч 3 (200 точек)
Загружен батч 4 (200 точек)
Загружен батч 5 (200 точек)
Загружен батч 6 (200 точек)
Загружен батч 7 (200 точек)
Загружен батч 8 (200 точек)
Загружен батч 9 (200 точек)
Загружен батч 10 (200 точек)
Загружен батч 11 (200 точек)
Загружен батч 12 (200 точек)
Загружен батч 13 (200 точек)
Загружен батч 14 (200 точек)
Загружен батч 15 (200 точек)
Загружен батч 16 (200 точек)
Загружен батч 17 (200 точек)
Загружен батч 18 (200 точек)
Загружен батч 19 (200 точек)
Загружен батч 20 (11 точек)


In [12]:
# Получить информацию о коллекции, включая количество точек
collection_info = client.get_collection(collection_name=COLLECTION_NAME)
print("Количество точек:", collection_info.points_count)

Количество точек: 3811
